In [2]:
import pandas as pd
import dask.dataframe as dd
import pyarrow as pa
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_for_model.parquet')

In [4]:
df.head()

,game_id,commence_time,time_pulled,home_team,away_team,barstool_1_odds,barstool_1_time,barstool_2_odds,barstool_2_time,betclic_1_odds,...,number_of_game_today,day_of_week,away_team_league,away_team_game_number,home_team_league,home_team_game_number,day_night,park_id,team_1,team_2
0,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:55:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:49:00,2.3,2022-11-05 16:49:00,1.68,...,0,Sat,NL,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies
1,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:45:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:44:00,2.3,2022-11-05 16:44:00,1.68,...,0,Sat,NL,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies
2,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:40:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:40:00,2.3,2022-11-05 16:40:00,1.68,...,0,Sat,NL,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies
3,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:35:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:35:00,2.3,2022-11-05 16:35:00,1.68,...,0,Sat,NL,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies
4,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:30:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:29:00,2.3,2022-11-05 16:29:00,1.68,...,0,Sat,NL,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies


In [5]:
# Read the column names 
all_columns = df.columns
columns = all_columns[5:]
info_columns = ['game_id', 'commence_time', 'time_pulled', 'home_team', 'away_team', 'team_1', 'team_2']

odds_columns = [x for x in columns if x.endswith('_odds')]

time_columns = [x for x in columns if x.endswith('_time')]

categorical_columns = ['day_of_week', 'away_team_league', 'home_team_league', 'day_night', 'park_id']

numerical_columns = ['number_of_game_today', 'away_team_game_number', 'home_team_game_number',]

In [6]:
# See the length of the df
len(df)

882515

In [7]:
# Assuming 'df' is your DataFrame
empty_row_counts = df.isna().sum()

In [8]:
empty_row_counts.to_csv('/Users/stefanfeiler/Desktop/empty_counts.csv')

There are a few books that are missing nearly allllll of the data.... need to think about how to handle these cases. 
It's nearly 40% of our dataset....

In [9]:
# Replace all the missing odds values with 0's
for col in odds_columns:
    df[col] = df[col].replace(np.nan, 0)
    df[col] = df[col].astype('float64')

In [10]:
# Replace all the missing time values with Jan 1, 1970
for col in time_columns:
    df[col] = df[col].replace(np.nan, '1/1/1970 00:00:00')

In [11]:
# See the amount of missing values in a few columns
percent_list = []
for col in odds_columns:
    zero_count = df[col].value_counts()[0]
    #print(f"{col}: {(zero_count/len(data_pd))*100}")
    percent_list.append((zero_count/len(df))*100)
print(np.array(percent_list).mean())

percent_list = []
for col in time_columns:
    zero_count = df[col].value_counts()[0]
  #print(f"{col}: {(zero_count/len(data_pd))*100}")
    percent_list.append((zero_count/len(df))*100)
print(np.array(percent_list).mean())

# oh my... 41% of our data set is empty... Need to read up on what to do in this situation....

41.210590304486736
41.210590304486736


# HERE ON OUT IS ONLY THE VERY FIRST TEST SO I CAN SEE HOW IT DOES ON SHITTY DATA

In [12]:
# Check length of Time since last update for each odds update 
# First we need to convert each time column as a pd.datetime
# With the missing values, we need to see how this time variable will be used further down the road. If we're planning on turning the last update time into a feature that measures difference between that time and the time_pulled or time_since_game_start variable, then we might want that final variable to be negative if there's no value so that the activation never turns on for that feature node in those instances. In that case, we should replace missing vaues with a time value way in the past. So lets do that. 
#time_cols = [x for x in columns if x.endswith('_time')]
for col in time_columns:
    df[col] = pd.to_datetime(df[col])

In [13]:
# This takes about 2 minutes

# We need to make the 'snapshot_time' variable which is the max value of the time columns by row
# find max value in each row and set it as the value of 'snapshot_time' column
df['snapshot_time_taken'] = df[time_columns].apply(lambda x: max(x), axis=1)

In [14]:
# We need to use that column as the time_pulled variable rather than the actual time pulled 
df['snapshot_time_taken'] = pd.to_datetime(df['snapshot_time_taken'])
df['commence_time'] = pd.to_datetime(df['commence_time'])

In [15]:
df['minutes_since_commence'] = (df['snapshot_time_taken'] - df['commence_time']).dt.total_seconds()/60

In [16]:
df['hour_of_start'] = df['commence_time'].dt.hour

In [ ]:
# new_data = df[df['minutes_since_commence'] >= -360]

In [17]:
new_data = df

In [18]:
len(new_data)

882515

In [19]:
new_data.head()

,game_id,commence_time,time_pulled,home_team,away_team,barstool_1_odds,barstool_1_time,barstool_2_odds,barstool_2_time,betclic_1_odds,...,away_team_game_number,home_team_league,home_team_game_number,day_night,park_id,team_1,team_2,snapshot_time_taken,minutes_since_commence,hour_of_start
0,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:55:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:49:00,2.3,2022-11-05 16:49:00,1.68,...,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies,2022-11-05 16:50:00,-14.0,17
1,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:45:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:44:00,2.3,2022-11-05 16:44:00,1.68,...,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies,2022-11-05 16:45:00,-19.0,17
2,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:40:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:40:00,2.3,2022-11-05 16:40:00,1.68,...,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies,2022-11-05 16:40:00,-24.0,17
3,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:35:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:35:00,2.3,2022-11-05 16:35:00,1.68,...,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies,2022-11-05 16:35:00,-29.0,17
4,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:30:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:29:00,2.3,2022-11-05 16:29:00,1.68,...,6.0,AL,6.0,N,HOU03,Houston Astros,Philadelphia Phillies,2022-11-05 16:30:00,-34.0,17


In [20]:
# Select the first subset:
cols_with_one = [col for col in new_data.columns if '_1' in col]
cols_with_two = [col for col in new_data.columns if '_2' in col]
extra_cols = ['game_id', 'commence_time', 'time_pulled', 'home_team', 'away_team', 'number_of_game_today', 'day_of_week', 'away_team_league', 'away_team_game_number', 'home_team_league', 'home_team_game_number', 'day_night', 'park_id', 'winning_team', 'minutes_since_commence', 'snapshot_time_taken', 'hour_of_start']

for each in extra_cols:
    cols_with_one.append(each)
    cols_with_two.append(each)

df1 = new_data[cols_with_one]
df2 = new_data[cols_with_two]

# # Get list of column names from df1
df1_cols = df1.columns.tolist()

# # Create dictionary to map column names in df2 to column names in df1
col_map = {col: df1_cols[i] for i, col in enumerate(df2.columns)}

# # Rename columns in df2 using dictionary
df2 = df2.rename(columns=col_map)

# Concatenate the subsets vertically
df_stacked = pd.concat([df1, df2], axis=0, ignore_index=True)

# Reset the index of the new DataFrame
df_stacked = df_stacked.reset_index(drop=True)

df_stacked['target'] = np.where(df_stacked['team_1'] == df_stacked['winning_team'], 1, 0)

df_stacked['this_team_league'] = np.where(df_stacked['team_1'] == df_stacked['home_team'], df_stacked['home_team_league'], df_stacked['away_team_league'])

df_stacked['opponent_league'] = np.where(df_stacked['team_1'] == df_stacked['home_team'], df_stacked['away_team_league'], df_stacked['home_team_league'])

df_stacked['this_team_game_of_season'] = np.where(df_stacked['team_1'] == df_stacked['home_team'], df_stacked['home_team_game_number'], df_stacked['away_team_game_number'])

df_stacked['opponent_game_of_season'] = np.where(df_stacked['team_1'] == df_stacked['home_team'], df_stacked['away_team_game_number'], df_stacked['home_team_game_number'])

df_stacked['home_away'] = np.where(df_stacked['team_1'] == df_stacked['home_team'], 1, 0)

df['snapshot_time'] = df['snapshot_time_taken'].dt.time

cols_to_drop=['commence_time', 'time_pulled', 'home_team', 'away_team', 'away_team_league', 'away_team_game_number','home_team_league', 'home_team_game_number', 'winning_team', 'snapshot_time_taken']

result = df_stacked.drop(columns=cols_to_drop)

# result_short = result.head(2000)

# result_short.to_csv('/Users/stefanfeiler/Desktop/result_short.csv')

In [ ]:
result.to_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_for_model.parquet')

In [21]:
len(result)

1765030

In [ ]:
result['number_of_game_today']